## Imports

In [1]:
import requests
import os

from urllib.request import urlopen
from zipfile import ZipFile

from os import listdir
from os.path import isfile, join

import pandas as pd
import glob

## Download and Unzip Data

In [2]:
def unzip(url, out_folder):
    try:
        tempzip = open("/tmp/tempfile.zip", "wb")
        tempzip.write(urlopen(url).read())
        tempzip.close()
        zf = ZipFile("/tmp/tempfile.zip")
        zf.extractall(path=out_folder)
        zf.close()
        print(f"Unzipped data to: {os.getcwd()}/{out_folder}")
    except Exception as req:
        raise UnzipFileError(req)

def get_data(url, out_folder):
    try:
        request = requests.get(url)
        if request.status_code == 200:
            if not os.path.exists(os.getcwd() + "/" + out_folder):
                os.makedirs(out_folder + "/")
        
    except Exception as req:
        print("Invalid URL provided.")
        print(req)

    return unzip(url, out_folder)

In [3]:
if not os.path.exists(os.getcwd() + "/data/raw/__MACOSX"):
    pass

else:
    files = glob.glob(os.getcwd() + "/data/raw/__MACOSX/*")
    for f in files:
        os.remove(f)
    os.rmdir(os.getcwd() + "/data/raw/__MACOSX")

if len(os.listdir(os.getcwd() + "/data/raw")) > 0:
    files = glob.glob(os.getcwd() + "/data/raw/*")
    for f in files:
        os.remove(f)
else:
    pass

In [4]:
time_get_data = %timeit -q -o -r 1 get_data(url="https://figshare.com/ndownloader/files/26766812", out_folder="data/raw")
print(f"Time Elapsed (to download and unzip the data): {time_get_data}")

Unzipped data to: /Users/vadim/block6/525/525-Group30/notebooks/data/raw
Unzipped data to: /Users/vadim/block6/525/525-Group30/notebooks/data/raw
Time Elapsed (to download and unzip the data): 7min 44s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Combining all .csv files into one DataFrame

In [5]:
path, dirs, files = next(os.walk("data/raw"))
file_count = len(files)

def combine_data():
    combined_df = pd.DataFrame()
    for file in files:
        temp = pd.DataFrame()
        model_name = file.split('_')[0]
        temp = pd.read_csv('data/raw/' + file)
        temp['model'] = model_name
        combined_df = pd.concat([combined_df, temp], axis=0)
    return combined_df

time_combine_data = %timeit -q -o -r 1 combine_data()
print(f"Time Elapsed (to combine the data): {time_combine_data}\n")

combined_df = combine_data()

combined_df.info()

Time Elapsed (to combine the data): 1min 37s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62513863 entries, 0 to 3541152
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float64
 2   lat_max        float64
 3   lon_min        float64
 4   lon_max        float64
 5   rain (mm/day)  float64
 6   model          object 
dtypes: float64(5), object(2)
memory usage: 3.7+ GB


In [6]:
if len(os.listdir(os.getcwd() + "/data/processed")) > 0:
    files = glob.glob(os.getcwd() + "/data/processed/*")
    for f in files:
        os.remove(f)

In [7]:
combined_df.to_csv('data/processed/combined_df.csv')

In [8]:
combined_df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
